In [51]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter
import statsmodels.formula.api as smf

In [52]:
city = 'MDE'
clientes = [348440135,348327939,348210549,348190834,348184917,348207832,348554432,348325658,348200398,347958887,346191386,346984549,233156998,346972916,347281869,347499131,347495129,346786621,346985225,346565576,346542628,346601719,343802000,345033627,345507655,345465178,345514277,339839704,344773594,338776228,344064096,346648085,260771383,341280383,341302262,335121430,331059072,327731858,324686381,324701972,324743234,346643631,345581588,346455377,346500638,346461542,346499245,345609005,346026344,345929577,329417640,147915852,345366492,264317973,344647971,343729478,343320253,343046761,317860369,316817910,310478266,300851465,300867085,298490793,343047958,340484519,339954423,339886271,337018760,336808200,336869603,329390069,288476817,288346626,286860391,286938894,286707052,285584288,283672813,277797878,312847372,302077944,298048512,294124524,290920171,286866030,284920694,284948903,277351019,271910254,194059150,231847535,177738257,144366491,258529629,124413564,236010040,151550498,348554403,348513616,348531755,348545115,348520698,348450051,348441074,348399341,348187938,348177329,199362861,240026445,203146892,212891407,191627180,118906295,120281611,348219763,348197791,348214882,193893841,167908364,234065957,240825420,165867892,228382013,252369428,240634101,169337384,118004173,210956731,222013424,125513925,180754525,256727024,120124645,264164895,125606279,245046559,226873118,232560365,150243822,241915625,122358854,237995842,241318995,257311425,241992232,161742385,141645801,157250586,224597560,231981049,128168258,120273406,152592581,140310591,187056210,263830616,263989814,259078176,249398681,257770487,152552893,172028771,217866653,265815070,265296843,225947431,209727534,128575291,181114827,124888698,229553418,134026708,147366082,158654958,136659599,119725960,198193349,168544006,208338961,215983806,194128240,259093782,146152745,150246563,219306596,236715565,152170783,183039544,118899059,142275847,227118180,243137877,248048723,182130732,268093605,151337296,199818082,138753543,150001060,163276000,265751804,190059556,133139497,138527983,325226162,175160768,127868589,210447140,229665732,162641660,347733111,347361370,346223519,346634714,327664416,327543322,326890292,210386207,121273788,128030650,125449581,121245799,167563049,126890733,253705590,148090043,184406300,204932622,215380726,240784353,238246861,198192547,216345306,215806019,222835634,203456835,134825669,263829546,329375395,346635309,347828815,347804470,136585015,219892179,216344342,267282550,268653549,345815845,345945824,346482659,346116603,345974346,345949837,120639273,134433029,304290635,303612509,229828752,127397368,156264118,346467101,308016015,305882484,305021764,132090530,198039668,176907154,244604708,158307383,132213410,172136532,124836825,347985819,346577991,346469489,346991171,347781023,347583816,347580476,286057867,347493840,347579381,341694553,346750133,345230705,346463488,346467286,346287106,346565602,345192535,344935699,344407030,343416819,347163160,346547633,346471324,346469727,345369330,345190255,344445420,344745066,343292587,338813157,338155526,338322842,33101,277885750,277304258,272651495,213038699,204162197,253702692,118394951,250456717,257797850,347276511,347531526,347846711,347781781,347767723,347961357,347128528,228472860,173056403,178086104,241205116,207309476,147379826
]
dias = 0
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde_query = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months

customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=200))#.strftime('%Y-%m-%d') #1 months

In [53]:
len(clientes)

329

In [54]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [55]:
df_check = get_fresh_query_result("https://internal-redash.federate.frubana.com/",165821,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [56]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    --AND extract(year from bo.submit_date::date) > 2021
    --AND extract(month from bo.submit_date::date) = 6--2021
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}' --NOT IN ('MTY','PBC','GDL')
    AND bo.customer_id IN {clientes}
)--,

--more AS (
SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    --DATE(bo.submit_date) AS submit_date,
    bs.addl_product_id AS card_id,
    bs2.name,
    COUNT(DISTINCT bo.order_id) AS frequency,
    SUM(boipd.quantity * foi.step_unit * boi.sale_price)*1.0 AS gmv_usd,
    SUM(boipd.quantity*foi.step_unit) AS demand

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"      boipd   ON boipd.order_item_id=boi.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"        boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
GROUP BY 1,2,3,4
""".format(clientes=tuple(set(clientes)), city = city)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["frequency", "gmv_usd","demand"]] = data_ventas_aux[["frequency", "gmv_usd","demand"]].astype('float32')
#data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

In [57]:
# Assuming your DataFrames are named 'df1' and 'df2'
data_ventas_aux = pd.merge(data_ventas_aux, df_check[['margin','card_id','clearence_days','fit_for_discount']], left_on='card_id', right_on='card_id', how='inner')

In [58]:
# Optional to clean the EagleEye
print(len(data_ventas_aux))
data_ventas_aux = data_ventas_aux.loc[data_ventas_aux.fit_for_discount != 'FALSE'].reset_index(drop=True).copy()
print(len(data_ventas_aux))

8642
5038


In [60]:
data_ventas_aux['cash_margin'] = data_ventas_aux.gmv_usd * data_ventas_aux.margin

In [68]:
# Assuming your DataFrame is named 'df'
grouped_df = data_ventas_aux.groupby('customer_id').apply(lambda x: x.nlargest(5, ['frequency', 'gmv_usd'])).reset_index(drop=True)

result_df = grouped_df.groupby('customer_id').agg({'card_id': list, 'name': list, 'frequency': list, 'gmv_usd': np.sum, 'cash_margin': np.sum}).reset_index()

result_df["margin"] = 100.00 * result_df.cash_margin / result_df.gmv_usd
result_df['margin'] = np.where(0.85*result_df.margin > 10, 10,0.85*result_df.margin)
result_df['margin'] = result_df['margin'].apply(lambda x: round(x, 2))
result_df['offer_name'] = ["SH_GRADS_"+city+"_"+todays_date+"_"+str(customer) for customer in result_df.customer_id]

In [70]:
# Step 1: Convert 'card_id' column from lists to individual values
df_exploded = result_df.explode('card_id')

# Step 2: Count the occurrences of each card ID
card_id_counts = df_exploded['card_id'].value_counts().reset_index()

# Rename columns and set 'card_id' as the index
card_id_counts.rename(columns={'index': 'card_id', 'card_id': 'frequency'}, inplace=True)

# Calculate cumulative percentage and percentage over the total
card_id_counts['cumulative_percentage'] = card_id_counts['frequency'].cumsum() / card_id_counts['frequency'].sum()
card_id_counts['percentage_over_total'] = card_id_counts['frequency'] / card_id_counts['frequency'].sum()

# Multiply by 100 and round to two decimal places
card_id_counts['cumulative_percentage'] = (card_id_counts['cumulative_percentage'] * 100).round(2)
card_id_counts['percentage_over_total'] = (card_id_counts['percentage_over_total'] * 100).round(2)

In [71]:
df_demand =  pd.merge(card_id_counts, data_ventas_aux[['card_id','name']], left_on='card_id', right_on='card_id', how='inner')
df_demand.drop_duplicates(subset=['card_id'], keep='first', inplace=True)

In [72]:
df_demand.head(10)

,card_id,frequency,cumulative_percentage,percentage_over_total,name
0,429315,141,9.13,9.13,Aceite Naoli 3LT
184,282287,103,15.80,6.67,Azúcar en Bulto x 50kg
300,280159,77,20.79,4.99,Cebolla Cabezona Blanca
462,282773,70,25.32,4.53,Gaseosa Coca Cola 12und x 400 ml
586,282279,54,28.82,3.50,Arroz Roa 25und x 500 gr
681,451960,53,32.25,3.43,Aceite de Palma Oleococina Bidon x 20 Lt
760,282610,46,35.23,2.98,Harina la Nieve x 50kg
817,282581,43,38.02,2.78,Harina Haz de Oros x 50kg
879,281379,43,40.80,2.78,Servilleta Popular x 600 und
969,282263,32,42.88,2.07,Aceite Naoli x 20Lt


In [73]:
def concatenate_skus(card_id_list):
    result = []
    for sku in card_id_list:
        try:
            sku_value = frida_products.loc[frida_products['id_tarjeta'] == sku, 'skus'].values[0]
            result.append(sku_value)
        except IndexError:
            result.append('')
    return '#'.join(result)

In [74]:
result_df.head(2)

,customer_id,card_id,name,frequency,gmv_usd,cash_margin,margin,offer_name
0,118004173,"[282610, 280159, 281468, 281463, 282279]","[Harina la Nieve x 50kg, Cebolla Cabezona Blan...","[9.0, 3.0, 2.0, 1.0, 1.0]",3389440.00,222712.361040,5.26,SH_GRADS_MDE_2023-07-18_118004173
1,118394951,"[282754, 282287, 282776, 282771, 282279]","[Gaseosa Quatro 12und x 400 ml, Azúcar en Bult...","[10.0, 8.0, 8.0, 7.0, 6.0]",2297215.75,156289.169813,5.44,SH_GRADS_MDE_2023-07-18_118394951


In [75]:
csv_file = {} 
csv_file['offer_name'] = result_df.offer_name
csv_file['offer_description'] = None
csv_file['discount'] = result_df.margin
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=dias)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(dias+1))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = 0
csv_file['max_uses_per_customer'] = 1
csv_file['customer_segment_id'] = [1]*len(result_df.offer_name) #[None]
csv_file['sku_id'] = result_df['card_id'].apply(concatenate_skus)

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [76]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_GRADS_MDE_2023-07-18_118004173,None,5.26,true,18-07-2023,20-07-2023,0,1,1,367142#631073#363439#365474#365462#366840
1,SH_GRADS_MDE_2023-07-18_118394951,None,5.44,true,18-07-2023,20-07-2023,0,1,1,367559#366859#367478#367466#366840
2,SH_GRADS_MDE_2023-07-18_118899059,None,10.00,true,18-07-2023,20-07-2023,0,1,1,608005#631073#363439#367084#441601#371215
3,SH_GRADS_MDE_2023-07-18_118906295,None,4.51,true,18-07-2023,20-07-2023,0,1,1,366859#367142#367478#367101#561459
4,SH_GRADS_MDE_2023-07-18_119725960,None,10.00,true,18-07-2023,20-07-2023,0,1,1,631073#363439#366556#364554#367045#604425


In [176]:
global_segment_list = []
for i in result_df.values:
    global_segment_list.append({'name':i[-1], 'customersIds': [i[0]]})

In [177]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [178]:
response.text

'Customer segments were created: [206751-SH_GRADS_MDE_2023-07-14_117965447, 206752-SH_GRADS_MDE_2023-07-14_117981322, 206753-SH_GRADS_MDE_2023-07-14_117990009, 206754-SH_GRADS_MDE_2023-07-14_118004173, 206755-SH_GRADS_MDE_2023-07-14_118027711, 206758-SH_GRADS_MDE_2023-07-14_118390825, 206756-SH_GRADS_MDE_2023-07-14_118390888, 206757-SH_GRADS_MDE_2023-07-14_118394951, 206759-SH_GRADS_MDE_2023-07-14_118527738, 206764-SH_GRADS_MDE_2023-07-14_118559961, 206760-SH_GRADS_MDE_2023-07-14_118887501, 206761-SH_GRADS_MDE_2023-07-14_118899059, 206762-SH_GRADS_MDE_2023-07-14_118906295, 206763-SH_GRADS_MDE_2023-07-14_118919772, 206765-SH_GRADS_MDE_2023-07-14_119625319, 206766-SH_GRADS_MDE_2023-07-14_119644027, 206767-SH_GRADS_MDE_2023-07-14_119660804, 206769-SH_GRADS_MDE_2023-07-14_119725960, 206768-SH_GRADS_MDE_2023-07-14_119759145, 206770-SH_GRADS_MDE_2023-07-14_119768054, 206771-SH_GRADS_MDE_2023-07-14_120056146, 206772-SH_GRADS_MDE_2023-07-14_120110923, 206773-SH_GRADS_MDE_2023-07-14_120124645, 

In [179]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [180]:
for i in global_offer_csv.offer_name:
    for j,z in sgt:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j
            break

In [182]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_GRADS_MDE_2023-07-14_117965447,None,10.00,true,14-07-2023,16-07-2023,0,1,206751,420017#420019#417542#420017#420019#417542#3634...
1,SH_GRADS_MDE_2023-07-14_117981322,None,10.00,true,14-07-2023,16-07-2023,0,1,206752,366859#631077#363480#363439#631073#631074#3635...
2,SH_GRADS_MDE_2023-07-14_117990009,None,10.00,true,14-07-2023,16-07-2023,0,1,206753,420017#420019#417542#420017#420019#417542#3990...
3,SH_GRADS_MDE_2023-07-14_118004173,None,5.44,true,14-07-2023,16-07-2023,0,1,206754,367142#363439#631073#363445#363530#365474
4,SH_GRADS_MDE_2023-07-14_118027711,None,10.00,true,14-07-2023,16-07-2023,0,1,206755,363474#367471#584675#363439#631073#616579


In [183]:
# Replace two or three '#' with one '#'
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('###', '#')
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('##', '#')
    
# Replace '#' at the beginning with an empty string
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('^#', '', regex=True)

# Remove rows where the string is empty
global_offer_csv = global_offer_csv.loc[global_offer_csv.discount > 0]
global_offer_csv = global_offer_csv.loc[global_offer_csv['sku_id'] != ""]

In [184]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/Below_Media/{city}/{todays_date}_SH_FARMING_{city}_OFFER_csv.csv', index = False)

In [185]:
result_df.to_excel(f'SD/export_data/Below_Media/{city}/{todays_date}_SH_RESULTS_FARMING_{city}_OFFER_csv.xlsx', index = False)